In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import pi, cos, sin, sqrt, atan2, ceil, floor
import random
from os.path import exists
from rtsvg import *
rt = RACETrack()
# Load the data
edges_filename  = '../../data/stanford/facebook/348.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df             = pl.DataFrame(_lu_)

_to_remove_ = set([428, 563, 1967]) # for 1912 graph
my_df = my_df.filter(~pl.col('fm').is_in(_to_remove_) & ~pl.col('to').is_in(_to_remove_))

# Find the high degree nodes
num_of_nodes_for_degree_threshold  = 16
g                 = rt.createNetworkXGraph(my_df, [('fm','to')])
degree_sorter     = []
for _node_ in g.nodes(): degree_sorter.append((g.degree[_node_], _node_))
degree_sorter.sort(reverse=True)
high_degree_nodes = set([_node_[1] for _node_ in degree_sorter[:num_of_nodes_for_degree_threshold]])

In [ ]:
# Find the communities
df_minus_high_degree_nodes = my_df.filter(~pl.col('fm').is_in(high_degree_nodes) & ~pl.col('to').is_in(high_degree_nodes))
g_minus_high_degree_nodes  = rt.createNetworkXGraph(df_minus_high_degree_nodes, [('fm','to')])
communities       = nx.community.louvain_communities(g_minus_high_degree_nodes)
community_lu      = {}
cluster_number    = max(max(my_df['fm']),max(my_df['to'])) + 1000 # needs to be different if nodes are strings
for _node_ in high_degree_nodes: 
    community_lu[cluster_number] = set([_node_])
    cluster_number += 1
for _community_ in communities:
    if len(_community_) <= 8: # if the community it too small, pull those nodes out
        for _node_ in _community_:
            community_lu[cluster_number] = set([_node_])
            cluster_number += 1
    else:
        community_lu[cluster_number] = set(_community_)
        cluster_number += 1

In [ ]:
# Collapse the communities
my_df_communities = rt.collapseDataFrameGraphByClusters(my_df, [('fm','to')], community_lu)
g_communities     = rt.createNetworkXGraph(my_df_communities, [('__fm__','__to__')])
communities_pos   = nx.spring_layout(g_communities)

_on_edge_ = ceil(sqrt(len(communities_pos.keys())))
_nodes_   = list(communities_pos.keys())
i         = 0
for x in range(_on_edge_):
    for y in range(_on_edge_):
        if i >= len(_nodes_): break
        # communities_pos[_nodes_[i]] = (x,y)
        i += 1

# Find the bounding box and determine the coordinate transforms
x_min, y_min, x_max, y_max = rt.positionExtents(communities_pos)
w, h         = 800, 800
cd_r         = 64
singleton_r  = 20
x_ins, y_ins = cd_r + 10, cd_r + 10
wxToSx       = lambda x: 1.5*cd_r + 5 + (w - 3*cd_r)*(x-x_min)/(x_max-x_min)
wyToSy       = lambda y: 1.5*cd_r + 5 + (h - 3*cd_r)*(y-y_min)/(y_max-y_min)

# Crunch the circles
communities_keys_ordered = sorted(list(community_lu.keys()))
circles                  = []
for i in range(len(communities_keys_ordered)):
    _community_key_ = communities_keys_ordered[i]
    sx, sy = wxToSx(communities_pos[_community_key_][0]), wyToSy(communities_pos[_community_key_][1])
    if len(community_lu[_community_key_]) == 1: circles.append((sx, sy, singleton_r))
    else:                                       circles.append((sx, sy, cd_r))

circles_adjusted  = rt.crunchCircles(circles)
#circles_adjusted = circles

circles_finalized = []

# And re-assign back into the pos array -- these again need to be normalized into the screen view
for_extent_calculation = {}
for i in range(len(circles_adjusted)): for_extent_calculation[i] = circles_adjusted[i][0], circles_adjusted[i][1]
x_min, y_min, x_max, y_max = rt.positionExtents(for_extent_calculation)
wxToSx       = lambda x: 1.5*cd_r + 5 + (w - 3*cd_r)*(x-x_min)/(x_max-x_min)
wyToSy       = lambda y: 1.5*cd_r + 5 + (h - 3*cd_r)*(y-y_min)/(y_max-y_min)
_pts_        = []
for i in range(len(communities_keys_ordered)):
    _community_key_ = communities_keys_ordered[i]
    sx,sy = wxToSx(circles_adjusted[i][0]), wyToSy(circles_adjusted[i][1])
    circles_finalized.append((sx, sy, circles_adjusted[i][2]))
    communities_pos[_community_key_] = (sx,sy)
    _pts_.append((sx,sy))

# Voronoi
voronoi_cells = rt.isedgarVoronoi(_pts_, Box=[(0,h),(w,h),(w,0),(0,0)])

In [ ]:

def shrinkCell(_cell_, _circle_, min_from_r=20.0):
    cx, cy, r = _circle_[0], _circle_[1], _circle_[2]
    # construct the perimeter that's the radius of the circle plus some margin
    _lines_ = []
    for i in range(len(_cell_)):
        x0,y0,x1,y1 = _cell_[i][0], _cell_[i][1], _cell_[(i+1)%len(_cell_)][0], _cell_[(i+1)%len(_cell_)][1]
        dx, dy      = x1-x0, y1-y0
        l           = sqrt(dx*dx + dy*dy)
        line_l      = max(w,h)
        dx, dy      = dx/l, dy/l
        pdx, pdy    = dy, -dx
        if pdx*(x1-cx) + pdy*(y1-cy) < 0: pdx, pdy = -dy, dx
        xp, yp = cx + pdx*(r+min_from_r), cy + pdy*(r+min_from_r)
        _lines_.append(((xp,yp),(xp+dx,yp+dy),(dx,dy)))
    # Find all line intersections
    _intersections_ = []
    for _line_ in _lines_:
        for _other_ in _lines_:
            if _line_ != _other_:
                _xy_ = rt.intersectionPoint(_line_, _other_)
                if _xy_ is not None:
                    _intersections_.append((_xy_, _line_, _other_))
    # Keep only the intersection points that occur before any other line segments are encountered
    _keepers_ = {}
    for _intersection_ in _intersections_:
        found_a_hit = False
        for _line_ in _lines_:
            if _line_ != _intersection_[1] and _line_ != _intersection_[2]:
                if rt.lineSegmentIntersectionPoint(_line_, (_intersection_[0],(cx,cy))) is not None:
                    found_a_hit = True
        if not found_a_hit:
            _keepers_[len(_keepers_)] = _intersection_[0]

    # Shrinkwrap the points
    _shrunken_ = rt.grahamScan(_keepers_)
    _as_pts_   = []
    for x in _shrunken_:
        _xy_ = _keepers_[x]
        _as_pts_.append(_xy_)
    return _as_pts_


In [ ]:
chord_diagram_num_entry_points = 4
chord_diagram_pushout          = 6
community_to_chord_diagram     = {}

svg = [f'<svg x="0" y="0" width="{w}" height="{h}"><rect x="0" y="0" width="{w}" height="{h}" fill="white" />']

# Render the communities as chord diagrams
dfs_rendered, _node_to_cd_, _cd_sxy_ = [], {}, {}
for _community_ in community_lu:
    _df_   = my_df.filter(pl.col('fm').is_in(community_lu[_community_]) & pl.col('to').is_in(community_lu[_community_]))
    dfs_rendered.append(_df_)
    if len(community_lu[_community_]) == 1:
        # sx, sy = wxToSx(communities_pos[_community_][0]), wyToSy(communities_pos[_community_][1])
        sx, sy = communities_pos[_community_][0], communities_pos[_community_][1]
        _node_ = list(community_lu[_community_])[0]
        _cd_sxy_[_node_] = (sx,sy)
        svg.append(f'<circle cx="{sx}" cy="{sy}" r="4" stroke="black" stroke-width="1.5" fill="black" />')
    else:
        _opacity_ = max(0.8 - len(community_lu[_community_])/100.0, 0.02)
        _cd_   = rt.chordDiagram(_df_, [('fm', 'to')], w=cd_r*2, h=cd_r*2, x_ins=0, y_ins=0, link_opacity=_opacity_, draw_border=False, node_h=4)
        community_to_chord_diagram[_community_] = _cd_
        for _node_ in community_lu[_community_]: _node_to_cd_[_node_] = _cd_
        # sx, sy = wxToSx(communities_pos[_community_][0]), wyToSy(communities_pos[_community_][1])
        sx, sy = communities_pos[_community_][0], communities_pos[_community_][1]
        _cd_sxy_[_cd_] = (sx,sy)
        svg.append(f'<g transform="translate({sx-cd_r}, {sy-cd_r})">{_cd_._repr_svg_()}</g>')

# Entry Points
entity_to_ins, entity_to_outs, entity_to_cdarc = {}, {}, {}
fm_arcs,       to_arcs                         = {}, {}       # collapsing lookups
arc_pos_and_vec                                = {}
community_list      = sorted(list(community_lu.keys()))
circles_for_routing = []
arc_str_to_circle_i = {}
for _community_i_ in range(len(community_list)): 
    _community_ = community_list[_community_i_]
    sx, sy = communities_pos[_community_][0], communities_pos[_community_][1]
    if len(community_lu[_community_]) == 1:
        circles_for_routing.append((sx,sy,singleton_r))
    else:
        circles_for_routing.append((sx,sy,cd_r))
for _community_i_ in range(len(community_list)):
    _community_ = community_list[_community_i_]
    if len(community_lu[_community_]) == 1:
        sx, sy             = communities_pos[_community_][0], communities_pos[_community_][1]
        _community_str_    = f'_community_{_community_i_}_'
        _community_fm_str_ = _community_str_ + 'fm_'
        _community_to_str_ = _community_str_ + 'to_'
        arc_str_to_circle_i[_community_fm_str_] = _community_i_
        arc_str_to_circle_i[_community_to_str_] = _community_i_
        fm_arcs[_community_fm_str_] = community_lu[_community_]
        to_arcs[_community_to_str_] = community_lu[_community_]
        _x_in_  = sx + (singleton_r + chord_diagram_pushout) * cos(0.0)
        _y_in_  = sy + (singleton_r + chord_diagram_pushout) * sin(0.0)
        arc_pos_and_vec[_community_fm_str_] = ((_x_in_, _y_in_), (cos(0.0), sin(0.0)), _community_i_)
        svg.append(f'<circle cx="{_x_in_}" cy="{_y_in_}" r="3" stroke="red" stroke-width="1.5" fill="none" />')
        _x_out_ = sx + (singleton_r + chord_diagram_pushout) * cos(pi)
        _y_out_ = sy + (singleton_r + chord_diagram_pushout) * sin(pi)
        arc_pos_and_vec[_community_to_str_] = ((_x_out_, _y_out_), (cos(pi), sin(pi)), _community_i_)
        svg.append(f'<circle cx="{_x_out_}" cy="{_y_out_}" r="3" stroke="green" stroke-width="1.5" fill="green" />')
    else:
        sx, sy      = communities_pos[_community_][0], communities_pos[_community_][1]
        _cd_        = community_to_chord_diagram[_community_]
        _cd_r_      = _cd_.r
        _angle_inc_ = 360.0 / chord_diagram_num_entry_points
        for i in range(chord_diagram_num_entry_points):
            _angle_min_            = _angle_inc_ * i
            _angle_max_            = _angle_inc_ * (i+1)
            _community_arc_str_    = f'_community_{_community_i_}_{i}_'
            _community_arc_fm_str_ = _community_arc_str_ + 'fm_'
            _community_arc_to_str_ = _community_arc_str_ + 'to_'
            arc_str_to_circle_i[_community_arc_fm_str_] = _community_i_
            arc_str_to_circle_i[_community_arc_to_str_] = _community_i_
            fm_arcs[_community_arc_fm_str_] = set()
            to_arcs[_community_arc_to_str_] = set()

            _angle_in_             = -_angle_inc_ / 8.0 + (_angle_min_ + _angle_max_) / 2.0
            _angle_out_            =  _angle_inc_ / 8.0 + (_angle_min_ + _angle_max_) / 2.0
            _x_in_                 = sx + (_cd_r_ + chord_diagram_pushout) * cos(_angle_in_ * pi / 180.0)
            _y_in_                 = sy + (_cd_r_ + chord_diagram_pushout) * sin(_angle_in_ * pi / 180.0)
            arc_pos_and_vec[_community_arc_fm_str_] = ((_x_in_,  _y_in_),  (cos(_angle_in_  * pi / 180.0), sin(_angle_in_  * pi / 180.0)), _community_i_) # (xy, uv, community_i)
            svg.append(f'<circle cx="{_x_in_}" cy="{_y_in_}" r="3" stroke="red" stroke-width="1.5" fill="none" />')

            _x_out_                = sx + (_cd_r_ + chord_diagram_pushout) * cos(_angle_out_ * pi / 180.0)
            _y_out_                = sy + (_cd_r_ + chord_diagram_pushout) * sin(_angle_out_ * pi / 180.0)
            arc_pos_and_vec[_community_arc_to_str_] = ((_x_out_, _y_out_), (cos(_angle_out_ * pi / 180.0), sin(_angle_out_ * pi / 180.0)), _community_i_) # (xy, uv, community_i)
            svg.append(f'<circle cx="{_x_out_}" cy="{_y_out_}" r="3" stroke="green" stroke-width="1.5" fill="green" />')

            _entities_             = _cd_.entitiesOnArc(_angle_min_, _angle_max_)
            for _entity_ in _entities_:
                entity_to_ins   [_entity_] = ((_x_in_,  _y_in_),  (cos(_angle_in_  * pi / 180.0), sin(_angle_in_  * pi / 180.0))) # (xy, uv)
                entity_to_outs  [_entity_] = ((_x_out_, _y_out_), (cos(_angle_out_ * pi / 180.0), sin(_angle_out_ * pi / 180.0))) # (xy, uv)
                entity_to_cdarc [_entity_] = _community_arc_str_
                fm_arcs[_community_arc_fm_str_].add(_entity_)
                to_arcs[_community_arc_to_str_].add(_entity_)
        
# Determine the edges between the communities and render them
df_rendered    = pl.concat(dfs_rendered)
df_remaining   = my_df.join(df_rendered, on=['fm', 'to'], how='anti')
df_collapsed   = rt.collapseDataFrameGraphByClustersDirectional(df_remaining, [('fm','to')], fm_arcs, to_arcs, color_by='to')
print(len(df_collapsed))
_max_          = df_collapsed['__count__'].max()
for i in range(len(df_collapsed)):
    _fm_, _to_, _count_ = df_collapsed['__fm__'][i], df_collapsed['__to__'][i], df_collapsed['__count__'][i]
    fm_xy = arc_pos_and_vec[_fm_][0]
    to_xy = arc_pos_and_vec[_to_][0]
    _stroke_width_ = 1.0 + (_count_ / _max_) * 3.0
    svg.append(f'<line x1="{fm_xy[0]}" y1="{fm_xy[1]}" x2="{to_xy[0]}" y2="{to_xy[1]}" stroke="{df_collapsed['__color__'][i]}" stroke-opacity="0.5" stroke-width="{_stroke_width_}" />')

# Render the cells -- shrinking them when necessary
for cell_i in range(len(voronoi_cells)):
    _circle_   = circles_finalized[cell_i]
    _cell_     = voronoi_cells[cell_i]
    _shrunken_ = shrinkCell(_cell_, _circle_)
    _path_     = f'M {_cell_[0][0]} {_cell_[0][1]} '
    for i in range(1, len(_cell_)): _path_ += f'L {_cell_[i][0]} {_cell_[i][1]} '
    _path_ += 'z'
    #svg.append(f'<path d="{_path_}" stroke="black" stroke-opacity="0.04" stroke-width="4.0" stroke-opacity="0.1" fill="none" />')
    _path_     = f'M {_shrunken_[0][0]} {_shrunken_[0][1]} '
    for i in range(1, len(_shrunken_)): _path_ += f'L {_shrunken_[i][0]} {_shrunken_[i][1]} '
    _path_ += 'z'
    #svg.append(f'<path d="{_path_}" stroke="black" stroke-opacity="0.5" stroke-width="4.0" fill="none" />')

def octagon(_x_, _y_, _r_): return [(_x_ + _r_ * cos((_angle_+22.5) * pi / 180.0), _y_ + _r_ * sin((_angle_+22.5) * pi / 180.0)) for _angle_ in range(0, 360, 45)]
for cell_i in range(len(circles_finalized)):
    _circle_  = circles_finalized[cell_i]
    _octagon_ = octagon(_circle_[0], _circle_[1], _circle_[2]+20)
    _path_    = f'M {_octagon_[0][0]} {_octagon_[0][1]} '
    for i in range(1, len(_octagon_)): _path_ += f'L {_octagon_[i][0]} {_octagon_[i][1]} '
    _path_ += 'z'
    svg.append(f'<path d="{_path_}" stroke="black" stroke-opacity="0.1" stroke-width="4.0" fill="none" />')

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#'''
svg_route = [f'<svg x="0" y="0" width="{w}" height="{h}"><rect x="0" y="0" width="{w}" height="{h}" fill="white" />']

def __formatTuple__(_tuple_): return (_tuple_[0][0], _tuple_[0][1], _tuple_[2])

for _to_, _to_df_ in df_collapsed.group_by('__to__'):
    _to_ = _to_[0]
    _color_ = rt.co_mgr.getColor(_to_)
    _entry_pt_ = __formatTuple__(arc_pos_and_vec[_to_])
    _exit_pts_ = []
    for i in range(len(_to_df_)):
        _fm_, _count_ = _to_df_['__fm__'][i], _to_df_['__count__'][i]
        _exit_pts_.append(__formatTuple__(arc_pos_and_vec[_fm_]))
    _route_ = rt.circularPathRouter(_entry_pt_, _exit_pts_, circles_for_routing)
    for i in range(len(_route_[0])):
        _coords_ = _route_[0][i]
        d =f'M {_coords_[0][0]} {_coords_[0][1]} '
        for k in range(1, len(_coords_)): d += f'L {_coords_[k][0]} {_coords_[k][1]} '
        svg_route.append(f'<path d="{d}" fill="none" stroke="{_color_}" stroke-width="2.0" />')


for i in range(len(circles_for_routing)):
    _circle_ = circles_for_routing[i]
    svg_route.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" stroke="black" stroke-width="4.0" fill="none" />')

svg_route.append('</svg>')
rt.tile([''.join(svg_route)])
#'''

In [ ]:
_lu_ = {'node_key':[], 'x':[], 'y':[], 'x_circle':[], 'y_circle':[], 'r_circle':[]}
for k in arc_pos_and_vec:
    _tuple_    = arc_pos_and_vec[k]
    _xy_       = _tuple_[0]
    _uv_       = _tuple_[1]
    _circle_i_ = _tuple_[2]
    _circle_   = circles_for_routing[_circle_i_]
    _lu_['node_key'].append(k)
    _lu_['x'].append(_xy_[0])
    _lu_['y'].append(_xy_[1])
    _lu_['x_circle'].append(_circle_[0])
    _lu_['y_circle'].append(_circle_[1])
    _lu_['r_circle'].append(_circle_[2])
df_arc_pos = pl.DataFrame(_lu_)
df_collapsed.write_parquet(edges_filename + '_collapsed_edges.14.parquet')
df_arc_pos  .write_parquet(edges_filename + '_arc_pos.14.parquet')